# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [190]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [191]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
   # raise NotImplementedError()
    test_df = nhl_df[['team','W','L','year']]
    test_df = test_df[test_df['year']==2018]
    test_df=test_df.drop([0,9,18,26])
    test_df['W/L Ratio'] = test_df['W'].astype(int)/(test_df['W'].astype(int)+test_df['L'].astype(int))
 

    replacements = {
    r'(.+)\*': r'\1'
    }
    
    test_df = test_df.replace(replacements, regex=True)
    test_df['team_name'] = test_df['team'].str.extract(r'(\w+)\b$')
    
    
    team_replacements = {
    r'Wings': r'Red Wings',
    r'Jackets': r'Blue Jackets',
    r'Leafs': r'Maple Leafs',
    r'Knights': r'Golden Knights',
    }
    test_df['team_name'] = test_df['team_name'].replace(team_replacements, regex=True)
    test_df['team_city'] = test_df['team'].str.replace('|'.join(test_df['team_name']),'').str.strip()
    #print(test_df)

    
    
    city_replacements = {
    r'Washington': r'Washington, D.C.',
    r'New York': r'New York City',
    r'New Jersey': r'New York City',
    r'Dallas': r'Dallas–Fort Worth',
    r'Minnesota': r'Minneapolis–Saint Paul',
    r'Vegas': r'Las Vegas',
    r'Tampa Bay': r'Tampa Bay Area',
    r'Arizona': r'Phoenix',
    r'Colorado': r'Denver',
    r'Carolina': r'Raleigh',
    r'Florida': r'Miami–Fort Lauderdale',
    r'San Jose': r'San Francisco Bay Area',
    r'Anaheim': r'Los Angeles'
    }
    
    
    cities_df= cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    cities_df= cities_df[cities_df['NHL'] != '—']
    test_df['team_city']= test_df['team_city'].replace(city_replacements, regex=True)
    cities_df = cities_df[~cities_df['NHL'].str.match(r'\[note \d+\]')]
    
    merge = pd.merge(test_df, cities_df, how='right', left_on='team_city', right_on='Metropolitan area')
    
    merge=merge[merge['team_name']!='Devils']
    merge=merge[merge['team_name']!='Islanders']
    merge=merge[merge['team_name']!='Ducks']
    merge.loc[merge["Metropolitan area"] == "New York City", "W/L Ratio"] = 0.5182013333333334 # mean of NY W-L%
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W/L Ratio"] = 0.6228945
    #return test_df#merge
    

    
    #return merge[['Metropolitan area','team_name','Population (2016 est.)[8]']].shape #cities["Metropolitan area"]
    population_by_region = merge['Population (2016 est.)[8]'].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L Ratio'].astype(float) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
#nhl_correlation()

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [225]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    test_df = nba_df[nba_df['year']==2018]
    test_df = test_df[['team','W','L']]
    
    
    replacements = {
    r'([\d\w ]+)[\W\d]*': r'\1'
    }
    city_replacements = {
    r'Washington': r'Washington, D.C.',
    r'New York': r'New York City',
    r'Brooklyn': r'New York City',
    r'Dallas': r'Dallas–Fort Worth',
    r'Minnesota': r'Minneapolis–Saint Paul',
    r'Miami': r'Miami–Fort Lauderdale',
    r'Golden State':r'San Francisco Bay Area',
    r'Utah':r'Salt Lake City',
    r'Indiana':r'Indianapolis',
    r'Portland Trail':r'Portland'
    }
    
    
    test_df['team'] = test_df['team'].replace(replacements, regex=True)
    test_df['team_name'] = test_df['team'].str.extract(r'(\w+)\b$')
    test_df['team_city'] = test_df['team'].str.replace('|'.join(test_df['team_name']),'').str.strip()
    test_df['team_city']= test_df['team_city'].replace(city_replacements, regex=True)
    test_df['W/L Ratio'] = test_df['W'].astype(int)/(test_df['W'].astype(int)+test_df['L'].astype(int))
    
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
    cities_df= cities_df[cities_df['NBA'] != '—']
    
    merge = pd.merge(test_df, cities_df, how='right', left_on='team_city', right_on='Metropolitan area').dropna()
    merge=merge[merge['team_name']!='Nets']
    merge=merge[merge['team_name']!='Clippers']
    merge.loc[merge["Metropolitan area"] == "New York City", "W/L Ratio"] = 0.3475
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W/L Ratio"] = 0.4695
    
    
    #return merge[['team_name','team_city','Metropolitan area','W/L%','NBA','Population (2016 est.)[8]']].reset_index()
    
    population_by_region = merge['Population (2016 est.)[8]'].astype('float') # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L Ratio'].astype('float') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.176630174061996

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [258]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    test_df = mlb_df[mlb_df['year']==2018]
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
    cities_df= cities_df[cities_df['MLB'] != '—']
    test_df['team_name'] = test_df['team'].str.extract(r'(\w+)\b$')
    test_df['team_city'] = test_df['team'].str.replace('|'.join(test_df['team_name']),'').str.strip()
    
    city_replacements = {
    r'Washington': r'Washington, D.C.',
    r'New York': r'New York City',
    r'San Francisco': r'San Francisco Bay Area',
    r'Oakland': r'San Francisco Bay Area',
    r'Texas': r'Dallas–Fort Worth',
    r'Minnesota': r'Minneapolis–Saint Paul',
    r'Miami': r'Miami–Fort Lauderdale',
    r'Colorado':r'Denver',
    r'Arizona':r'Phoenix',
    r'Indiana':r'Indianapolis',
    r'Boston Red':r'Boston',
    r'Chicago White':r'Chicago',
    r'Toronto Blue':r'Toronto',
    r'Tampa Bay':r'Tampa Bay Area',
    }
    test_df['team_city']= test_df['team_city'].replace(city_replacements, regex=True)
    
    merge = pd.merge(test_df,cities_df, how='outer', left_on='team_city', right_on='Metropolitan area').dropna()
    merge['W/L Ratio'] = merge['W'].astype(int)/(merge['W'].astype(int)+merge['L'].astype(int))
    
    merge=merge[merge['team_name']!='Cubs']
    merge=merge[merge['team_name']!='Mets']
    merge=merge[merge['team_name']!='Giants']
    merge=merge[merge['team_name']!='Dodgers']
    merge.loc[merge["Metropolitan area"] == "New York City", "W/L Ratio"] = 0.5462965
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W/L Ratio"] = 0.529122
    merge.loc[merge["Metropolitan area"] == "San Francisco Bay Area", "W/L Ratio"] = 0.524691
    merge.loc[merge["Metropolitan area"] == "Chicago", "W/L Ratio"] = 0.482769
    
    
    #return merge
    population_by_region = merge['Population (2016 est.)[8]'].astype('float') # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L Ratio'].astype('float') # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15027709337426867

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [285]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    test_df = nfl_df[nfl_df['year']==2018]
    cities_df = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
    cities_df= cities_df[cities_df['NFL'] != '—']
    test_df = test_df.drop([0,5,10,15,20,25,30,35])
    test_df['W/L Ratio'] = test_df['W'].astype(int)/(test_df['W'].astype(int)+test_df['L'].astype(int))

    
    replacements = {
    r'([\d\w ]+)[\W]*': r'\1'
    }
    
    city_replacements = {
    r'Washington': r'Washington, D.C.',
    r'New York': r'New York City',
    r'San Francisco': r'San Francisco Bay Area',
    r'Oakland': r'San Francisco Bay Area',
    r'Dallas': r'Dallas–Fort Worth',
    r'Minnesota': r'Minneapolis–Saint Paul',
    r'Miami': r'Miami–Fort Lauderdale',
    r'New England':r'Boston',
    r'Arizona':r'Phoenix',
    r'Tampa Bay':r'Tampa Bay Area',
    r'Tennessee':r'Nashville',
    r'Carolina':r'Charlotte',
    }
    
    
    test_df['team'] = test_df['team'].replace(replacements, regex=True)
    test_df['team_name'] = test_df['team'].str.extract(r'(\w+)\b$')
    test_df= test_df[['team','team_name','W/L Ratio']]
    
    test_df['team_city'] = test_df['team'].str.replace('|'.join(test_df['team_name']),'').str.strip()
    test_df['team_city']= test_df['team_city'].replace(city_replacements, regex=True)
    
    merge = pd.merge(test_df, cities_df, how='outer', left_on='team_city', right_on='Metropolitan area').dropna()
    
    merge=merge[merge['team_name']!='Raiders']
    merge=merge[merge['team_name']!='Giants']
    merge=merge[merge['team_name']!='Rams']
    merge.loc[merge["Metropolitan area"] == "New York City", "W/L Ratio"] = 0.28125
    merge.loc[merge["Metropolitan area"] == "Los Angeles", "W/L Ratio"] = 0.78125
    merge.loc[merge["Metropolitan area"] == "San Francisco Bay Area", "W/L Ratio"] = 0.25
    
    #return merge
    population_by_region = merge['Population (2016 est.)[8]'].astype('float') # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L Ratio'].astype('float') # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004922112149349412

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values